In [1]:
import numpy as np

f2py works nicely with numpy arrays, passing them as (multi-)dimensional arrays to Fortran procedures.  However, there are a few caveats.  Consider the following Fortran module.

In [24]:
%pycat matrix_mod.f90

module matrix_mod
    use, intrinsic :: iso_fortran_env, only : error_unit
    implicit none

    private
    public :: norm_matrix_rows

contains

    subroutine norm_matrix_rows(matrix)
        implicit none
        integer, parameter :: DP = selected_real_kind(15, 307)
        real(kind=DP), dimension(:, :), intent(inout) :: matrix
        real(kind=DP), dimension(:), allocatable :: row_sums
        integer :: status, j

        allocate (row_sums(size(matrix, 1)), stat=status)
        if (status /= 0) then
            write (unit=error_unit, fmt='(A)') 'error: can not allocate row sums'
            stop 11
        end if
        row_sums = sum(matrix, dim=2)
        do j = 1, size(matrix, 2)
            matrix(:, j) = matrix(:, j)/row_sums
        end do
        deallocate (row_sums)
    end subroutine norm_matrix_rows

end module matrix_mod


The subroutine `norm_matrix_rows` takes a 2-dimensional array and normalizes it row-wise.  After calling the subroutine, the sum of each row is 1.

Note that the subroutine takes a double precision matrix as argument.  Strangely (and annoyingly), the kind has to be defined using a parameter, rather than using `iso_fotran_env`.

We can compile this using f2py, creating a Python module `matrices`.

In [12]:
!f2py -c -m matrices matrix_mod.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "matrices" sources
f2py options: []
f2py:> /tmp/tmpofepkdk9/src.linux-x86_64-3.8/matricesmodule.c
creating /tmp/tmpofepkdk9/src.linux-x86_64-3.8
Reading fortran codes...
	Reading file 'matrix_mod.f90' (format:free)
{'before': '', 'this': 'use', 'after': ', intrinsic :: iso_fortran_env, only : error_unit '}
Line #2 in matrix_mod.f90:"    use, intrinsic :: iso_fortran_env, only : error_unit "
	analyzeline: Could not crack the use statement.
Post-processing...
	Block: matrices
			Block: matrix_mod
				Block: norm_matrix_rows
Post-processing (stage 2)...
	Block: matrices
		Block: unknown_interface
			Block: matrix_mod
				Block: norm_matrix_rows
Building modules...
	Building module "matrices"...
		Constructing F90 module support

Import the Fortran module from the Python module.

In [13]:
from matrices import matrix_mod

Create a 2-dimensional array, note that it has to be stored in column-major format, which is not numpy's default  format.

In [19]:
A = np.asfortranarray(np.arange(1.0, 12.5, 1.0, ).reshape((3, 4)))

In [20]:
A

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.],
       [ 9., 10., 11., 12.]])

The type of the numpy array is `np.float64` which will be translated to the expected kind in Fortran.

In [21]:
A.dtype

dtype('float64')

We can call the Fortran subroutine.

In [22]:
matrix_mod.norm_matrix_rows(A)

In [23]:
A

array([[0.1       , 0.2       , 0.3       , 0.4       ],
       [0.19230769, 0.23076923, 0.26923077, 0.30769231],
       [0.21428571, 0.23809524, 0.26190476, 0.28571429]])

We can verify the result easily.

In [25]:
A.sum(axis=1)

array([1., 1., 1.])